The goal is to scrape data related to derivatives of cryptocurrencies markets in order to find patterns, gather data over the course of time with the ultimate goal of training ML models for predictions purposes.

The main cryptocurrencies Derivatives data are 1/ Funding Rates 2/ Open Interest of Futures & Options.

Positive funding rates are indicative of a majority of bullish traders holding long positions who pay funding to short traders.
Negative funding rates are indicative of majority of short sellers who in turn pay funding to long traders.

Funding rates below 0.01 % are indicative of a bullish market.
Funding rates at 0.01 are considered neutral
Funding rates above 0.01 are indicative of a bearish market.

In [2]:
from requests_html import HTMLSession
import pandas as pd

url = 'https://defirate.com/funding/'
s = HTMLSession()
r = s.get(url)

# r.html.render(sleep = 3)
# print(r.status_code)

products = r.html.xpath('//*[@id="main-table"]', first = True)
print(products.text)

with open("funding_rates.txt", "w") as f:
    f.write(products.text)

Binance
Trade
dYdX
Trade
FTX
BitMEX
BTC
-0.008009%
{ "@context":"http://schema.org", "@type":"LoanOrCredit", "name":"Binance BTC Loan", "loanTerm":{ "@type":"QuantitativeValue", "value":"365", "unitCode": "DAY" }, "annualPercentageRate":[ { "@type":"QuantitativeValue", "name": "variable interest rates", "value":"-0.008009" } ], "amount":[ { "@type": "MonetaryAmount", "name":"BTC Loan", "value":"1000", "currency":"BTC" } ] }
-0.01119944%
{ "@context":"http://schema.org", "@type":"LoanOrCredit", "name":"dYdX BTC Loan", "loanTerm":{ "@type":"QuantitativeValue", "value":"365", "unitCode": "DAY" }, "annualPercentageRate":[ { "@type":"QuantitativeValue", "name": "variable interest rates", "value":"-0.01119944" } ], "amount":[ { "@type": "MonetaryAmount", "name":"BTC Loan", "value":"1000", "currency":"BTC" } ] }
-0.0072%
{ "@context":"http://schema.org", "@type":"LoanOrCredit", "name":"FTX BTC Loan", "loanTerm":{ "@type":"QuantitativeValue", "value":"365", "unitCode": "DAY" }, "annualPercenta

Converting the Data saved in .txt into a CSV for Optimal Data Cleaning.

In [3]:
# pip install xlwt

In [4]:
#Reading file & creating dataframe in a csv
import time
import xlwt
import csv

date_string = time.strftime('%Y-%m-%d_%H.%M')
with open('funding_rates.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split("\n") for line in stripped if line)

    #Adding Date + Time to our CSV filename

    with open("funding_rates_"+ date_string+ ".csv", "w") as out_file:
        writer = csv.writer(out_file)
        writer.writerows(('title', 'intro'))
        writer.writerows(lines)
        df =out_file

df = pd.read_csv("funding_rates_"+date_string+".csv", encoding = "ISO-8859-1")
df

,t,i,t.1,l,e
0,i,n,t,r,o
1,Binance,NaN,NaN,NaN,NaN
2,Trade,NaN,NaN,NaN,NaN
3,dYdX,NaN,NaN,NaN,NaN
4,Trade,NaN,NaN,NaN,NaN
...,...,...,...,...,...
1786,0.00770552%,NaN,NaN,NaN,NaN
1787,"{ ""@context"":""http://schema.org"", ""@type"":""Loa...",NaN,NaN,NaN,NaN
1788,-0.0248%,NaN,NaN,NaN,NaN
1789,"{ ""@context"":""http://schema.org"", ""@type"":""Loa...",NaN,NaN,NaN,NaN


Data Cleaning & Manipulation for better presentability \n
NOTE: df is  the dataframe the bot is working with

In [5]:
# Setting Pandas as to show all rows here 
pd.set_option('display.max_rows', df.shape[0]+1)

# Dropping needless rows at the top and emtpy columns i, t.1, l ,e
df = df.drop(labels=[1,2,3,4,5,6,], axis = 0)
df = df.drop(columns = ["i", "t.1", 'l', "e"], axis =1)
df.reset_index(drop=True)

# Renaming a column for better readability
df = df.rename(columns = {'t': "messed up data 2 clean"})

#Taking a look at our Dataframe
print(df.head(20))

                               messed up data 2 clean
0                                                   i
7                                                 BTC
8                                          -0.008009%
9   { "@context":"http://schema.org", "@type":"Loa...
10                                       -0.01119944%
11  { "@context":"http://schema.org", "@type":"Loa...
12                                           -0.0072%
13  { "@context":"http://schema.org", "@type":"Loa...
14                                            0.0043%
15  { "@context":"http://schema.org", "@type":"Loa...
16                                                ETH
17                                              0.01%
18  { "@context":"http://schema.org", "@type":"Loa...
19                                        0.01900792%
20  { "@context":"http://schema.org", "@type":"Loa...
21                                           -0.0016%
22  { "@context":"http://schema.org", "@type":"Loa...
23                          

In [6]:
# As you can see in the data within the "messed up data 2 clean" column main separator is a comma
# We rely on str.split with comma to split this columns amongst many to better process for cleaning 

df = df['messed up data 2 clean'].str.split(',', expand=True)

# We want the Exchange's name located in column 2 
            # the funding rate located in column 8
            # the cryptocurrency located in column 12.


#Dropping useless columns 
cols = [0,1,3,4,5,6,7,9,10,11]
df.drop(df.columns[cols], axis = 1, inplace = True)

#Dropping first empty rows
df = df.drop(labels=[0,7,8], axis = 0)

#Reordering columns for better readbility
#Renaming columns as aforementioned columns of interest 
df = df.reindex(columns=[12,8,2])
df = df.rename(columns = ({12: "Crypto", 8:'Funding Rate', 2:'Exchange' }))

df.head(20)


,Crypto,Funding Rate,Exchange
9,"""currency"":""BTC"" } ] }","""value"":""-0.008009"" } ]","""name"":""Binance BTC Loan"""
10,None,None,None
11,"""currency"":""BTC"" } ] }","""value"":""-0.01119944"" } ]","""name"":""dYdX BTC Loan"""
12,None,None,None
13,"""currency"":""BTC"" } ] }","""value"":""-0.0072"" } ]","""name"":""FTX BTC Loan"""
14,None,None,None
15,"""currency"":""BTC"" } ] }","""value"":""0.0043"" } ]","""name"":""BitMEX BTC Loan"""
16,None,None,None
17,None,None,None
18,"""currency"":""ETH"" } ] }","""value"":""0.01"" } ]","""name"":""Binance ETH Loan"""


In [7]:
df = df.dropna()
df.head(20)

,Crypto,Funding Rate,Exchange
9,"""currency"":""BTC"" } ] }","""value"":""-0.008009"" } ]","""name"":""Binance BTC Loan"""
11,"""currency"":""BTC"" } ] }","""value"":""-0.01119944"" } ]","""name"":""dYdX BTC Loan"""
13,"""currency"":""BTC"" } ] }","""value"":""-0.0072"" } ]","""name"":""FTX BTC Loan"""
15,"""currency"":""BTC"" } ] }","""value"":""0.0043"" } ]","""name"":""BitMEX BTC Loan"""
18,"""currency"":""ETH"" } ] }","""value"":""0.01"" } ]","""name"":""Binance ETH Loan"""
20,"""currency"":""ETH"" } ] }","""value"":""0.01900792"" } ]","""name"":""dYdX ETH Loan"""
22,"""currency"":""ETH"" } ] }","""value"":""-0.0016"" } ]","""name"":""FTX ETH Loan"""
24,"""currency"":""ETH"" } ] }","""value"":""0.0788"" } ]","""name"":""BitMEX ETH Loan"""
27,"""currency"":""XRP"" } ] }","""value"":""0.006868"" } ]","""name"":""Binance XRP Loan"""
30,"""currency"":""XRP"" } ] }","""value"":""-0.0144"" } ]","""name"":""FTX XRP Loan"""


In [8]:
#Verifying the data type of each column in df before further processing
df.dtypes

Crypto          object
Funding Rate    object
Exchange        object
dtype: object

All data in dataframe df is String Data type, so far.

In [9]:
df['Crypto'] = df['Crypto'].str[12:].str[:-5]
df['Funding Rate'] = df['Funding Rate'].str[10:].str[:-5]
df['Exchange'] = df['Exchange'].str[8:].str[:-5]

df.head(12)

,Crypto,Funding Rate,Exchange
9,"""BTC""",-0.008009,"""Binance BTC"
11,"""BTC""",-0.01119944,"""dYdX BTC"
13,"""BTC""",-0.0072,"""FTX BTC"
15,"""BTC""",0.0043,"""BitMEX BTC"
18,"""ETH""",0.01,"""Binance ETH"
20,"""ETH""",0.01900792,"""dYdX ETH"
22,"""ETH""",-0.0016,"""FTX ETH"
24,"""ETH""",0.0788,"""BitMEX ETH"
27,"""XRP""",0.006868,"""Binance XRP"
30,"""XRP""",-0.0144,"""FTX XRP"


In [10]:
df['Funding Rate'] = df['Funding Rate'].astype(float)
df['date_time'] = pd.to_datetime('now')


In [11]:
#Dataframe cleaned, with funding rate converted as a float 
df

,Crypto,Funding Rate,Exchange,date_time
9,"""BTC""",-0.008009,"""Binance BTC",2022-05-23 23:09:45.144393
11,"""BTC""",-0.011199,"""dYdX BTC",2022-05-23 23:09:45.144393
13,"""BTC""",-0.007200,"""FTX BTC",2022-05-23 23:09:45.144393
15,"""BTC""",0.004300,"""BitMEX BTC",2022-05-23 23:09:45.144393
18,"""ETH""",0.010000,"""Binance ETH",2022-05-23 23:09:45.144393
20,"""ETH""",0.019008,"""dYdX ETH",2022-05-23 23:09:45.144393
22,"""ETH""",-0.001600,"""FTX ETH",2022-05-23 23:09:45.144393
24,"""ETH""",0.078800,"""BitMEX ETH",2022-05-23 23:09:45.144393
27,"""XRP""",0.006868,"""Binance XRP",2022-05-23 23:09:45.144393
30,"""XRP""",-0.014400,"""FTX XRP",2022-05-23 23:09:45.144393


In [12]:
#Saving data with adding Date & Time
from datetime import datetime

date_string = time.strftime('%Y-%m-%d_%H.%M')

df.to_csv('Dataframes/DefiRate.com_Funding_Rates_'+ date_string+'.csv')
